In [145]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [146]:
import numpy as np
import pandas as pd

In [147]:
path = "C:/Users/mbr19/OneDrive/Escritorio/Ultimo semestre/Python/Segundo Corte/PROYECTO FINAL IA CORTE 2-20241012/PROYECTO FINAL IA CORTE 2/pixeles.xlsx"
matrix = pd.read_excel(path, header =None)
print(matrix.head())

   0  1  2  3  4  5
0  0  0  1  0  0  0
1  0  1  1  0  0  0
2  0  0  1  0  0  0
3  0  0  1  0  0  0
4  0  1  1  1  0  0


In [148]:
matrix = matrix.to_numpy()

In [149]:
matrix.shape

(6, 6)

In [150]:
##Convolusión

In [151]:
# Definir los kernels para detección de bordes
kernel_horizontal = np.array([[1, 1, 1], [0, 0, 0], [-1,-1,-1]])
kernel_vertical = np.array([[1, 0,-1], [1, 0,-1], [1, 0,-1]])


In [152]:
def apply_convolution(matrix, kernel_type="horizontal"):
    # Seleccionar el kernel
    if kernel_type == "horizontal":
        kernel = kernel_horizontal
    elif kernel_type == "vertical":
        kernel = kernel_vertical
    else:
        raise ValueError("El tipo de kernel debe ser 'horizontal' o 'vertical'.")

    # Inicializar la matriz de salida
    output = np.zeros((matrix.shape[0] - 2, matrix.shape[1] - 2))

    # Realizar la convolución
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            # Extraer la submatriz de 3x3
            sub_matrix = matrix[i:i+3, j:j+3]
            # Calcular la suma de la multiplicación del kernel con la submatriz
            output[i, j] = np.sum(sub_matrix * kernel)
    
    return output

In [153]:
output_horizontal = apply_convolution(matrix, kernel_type="horizontal")
print("Matriz de salida (bordes horizontales):\n", output_horizontal)

# Aplicar la convolución con el kernel vertical
output_vertical = apply_convolution(matrix, kernel_type="vertical")
print("Matriz de salida (bordes verticales):\n", output_vertical)

Matriz de salida (bordes horizontales):
 [[ 0.  0.  0.  0.]
 [ 1.  1.  0.  0.]
 [-1. -2. -1. -1.]
 [ 1.  1.  1.  0.]]
Matriz de salida (bordes verticales):
 [[-3.  1.  3.  0.]
 [-3.  1.  3.  0.]
 [-3.  0.  3.  1.]
 [-2.  0.  2.  1.]]


In [ ]:
### Padding

In [163]:
def add_padding(matrix, padding_rows, padding_cols):
    # Asegurarse de que la matriz es un array de numpy
    matrix = np.array(matrix)
    
    # Crear matrices de ceros con el tamaño adecuado
    row_padding = np.zeros((padding_rows, matrix.shape[1]))  # Padding de filas
    col_padding = np.zeros((matrix.shape[0] + padding_rows * 2, padding_cols))  # Padding de columnas
    
    # Concatenar las filas de ceros en la parte superior y en la parte inferior
    padded_matrix = np.vstack((row_padding, matrix, row_padding))
    
    # Concatenar las columnas de ceros a la izquierda y a la derecha
    padded_matrix = np.hstack((col_padding, padded_matrix, col_padding))
    
    return padded_matrix

In [165]:
padding_rows = 1  # Número de filas de ceros a agregar
padding_cols = 1  # Número de columnas de ceros a agregar
padded_matrix = add_padding(matrix, padding_rows, padding_cols)

print("Matriz original:\n", matrix)
print("\nMatriz con padding:\n", padded_matrix)

Matriz original:
 [[0 0 1 0 0 0]
 [0 1 1 0 0 0]
 [0 0 1 0 0 0]
 [0 0 1 0 0 0]
 [0 1 1 1 0 0]
 [0 0 0 0 0 0]]

Matriz con padding:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [167]:
###Stride

In [169]:
def apply_stride(matrix, kernel_type, stride):
    # Asegurarse de que la matriz es un array de numpy
    matrix = np.array(matrix)

    # Definir el kernel para la detección de bordes
    if kernel_type == "horizontal":
        kernel = np.array([[1, 1, 1], 
                           [0, 0, 0], 
                           [-1, -1, -1]])
    elif kernel_type == "vertical":
        kernel = np.array([[1, 0, -1], 
                           [1, 0, -1], 
                           [1, 0, -1]])
    else:
        raise ValueError("Tipo de kernel no válido. Usa 'horizontal' o 'vertical'.")

    # Dimensiones de la matriz original y de salida
    output_rows = (matrix.shape[0] - 2) // stride + 1
    output_cols = (matrix.shape[1] - 2) // stride + 1
    output = np.zeros((output_rows, output_cols))

    # Aplicar el kernel con el stride
    for i in range(0, matrix.shape[0] - 2, stride):
        for j in range(0, matrix.shape[1] - 2, stride):
            # Extraer la submatriz de 3x3
            sub_matrix = matrix[i:i + 3, j:j + 3]
            # Calcular la suma de la multiplicación del kernel con la submatriz
            output[i // stride, j // stride] = np.sum(sub_matrix * kernel)

    return output

In [171]:
stride = 2  # Magnitud de los saltos
output_horizontal = apply_stride(matrix, kernel_type="horizontal", stride=stride)
output_vertical = apply_stride(matrix, kernel_type="vertical", stride=stride)

print("Matriz de salida (bordes horizontales):\n", output_horizontal)
print("\nMatriz de salida (bordes verticales):\n", output_vertical)

Matriz de salida (bordes horizontales):
 [[ 0.  0.  0.]
 [-1. -1.  0.]
 [ 0.  0.  0.]]

Matriz de salida (bordes verticales):
 [[-3.  3.  0.]
 [-3.  3.  0.]
 [ 0.  0.  0.]]


In [173]:
##Stacking

In [179]:
def generate_kernels(n):
    # Genera n kernels aleatorios de 3x3
    return [np.random.randint(-1, 2, (3, 3)) for _ in range(n)]

def apply_convolution_with_kernel(matrix, kernel):
    # Aplica convolución con el kernel dado
    output = np.zeros((matrix.shape[0] - 2, matrix.shape[1] - 2))
    for i in range(output.shape[0]):
        for j in range(output.shape[1]):
            sub_matrix = matrix[i:i+3, j:j+3]
            output[i, j] = np.sum(sub_matrix * kernel)
    return output

def generate_feature_maps(matrix, n):
    # Genera n kernels y aplica convolución en horizontal y vertical para cada uno
    kernels = generate_kernels(n)
    feature_maps = []

    for kernel in kernels:
        horizontal_map = apply_convolution_with_kernel(matrix, kernel)  # Cambiado a apply_convolution_with_kernel
        vertical_map = apply_convolution_with_kernel(matrix, kernel.T)  # Transpuesta para detección vertical
        feature_maps.append((horizontal_map, vertical_map))  # Agregamos como un par
    
    return feature_maps

In [181]:
n = 3  # Número de mapas a generar
feature_maps = generate_feature_maps(matrix, n)

# Mostrar los mapas generados
for idx, feature_map in enumerate(feature_maps):
    print(f"Mapa de características {idx + 1}:\n{feature_map}\n")

Mapa de características 1:
(array([[-2.,  0.,  0.,  0.],
       [-4., -1.,  0.,  0.],
       [-4., -3., -1., -1.],
       [-1.,  0.,  2.,  1.]]), array([[-2.,  0., -2.,  0.],
       [-2.,  1., -2.,  0.],
       [-4., -1., -3., -1.],
       [-1.,  0.,  0., -1.]]))

Mapa de características 2:
(array([[ 0., -1., -1.,  0.],
       [-2., -1., -1.,  0.],
       [-2., -3., -2., -1.],
       [ 1., -1.,  1.,  0.]]), array([[-2., -1.,  0.,  0.],
       [-3.,  0.,  0.,  0.],
       [-4.,  0., -1.,  1.],
       [-1., -1.,  0., -1.]]))

Mapa de características 3:
(array([[1., 0., 1., 0.],
       [0., 0., 1., 0.],
       [2., 1., 2., 1.],
       [1., 0., 0., 0.]]), array([[ 2.,  0., -1.,  0.],
       [ 2.,  1., -1.,  0.],
       [ 3.,  1.,  0.,  0.],
       [ 2.,  0., -1., -1.]]))



In [ ]:
### Maxpooling

In [135]:
def max_pooling(matrix, stride=1):
    # Dimensiones de la matriz original
    rows, cols = matrix.shape
    
    # Calcular las dimensiones de la matriz de salida
    output_rows = (rows - 2) // stride + 1
    output_cols = (cols - 2) // stride + 1
    
    # Crear la matriz de salida
    output = np.zeros((output_rows, output_cols))
    
    # Recorrer la matriz con el kernel de 2x2
    for i in range(0, rows - 1, stride):
        for j in range(0, cols - 1, stride):
            # Extraer el bloque de 2x2
            block = matrix[i:i+2, j:j+2]
            # Calcular el valor máximo en el bloque
            output[i // stride, j // stride] = np.max(block)
    
    return output

In [137]:
# Aplicar max pooling con stride = 2
output = max_pooling(matrix, stride=2)
print("Matriz de salida después de Max Pooling:\n", output)

Matriz de salida después de Max Pooling:
 [[1. 1. 0.]
 [0. 1. 0.]
 [1. 1. 0.]]
